In [1]:
import boto3, argparse
import sagemaker
from sagemaker.estimator import Estimator
from sagemaker.experiments.run import load_run
from sagemaker.session import Session


from sagemaker.huggingface import HuggingFaceProcessor, HuggingFace
from sagemaker.workflow.pipeline_experiment_config import PipelineExperimentConfig

from sagemaker.workflow.execution_variables import ExecutionVariables

from smexperiments import tracker
from sagemaker.utils import unique_name_from_base

ModuleNotFoundError: No module named 'boto3'

In [4]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer


In [5]:
model_name = "distilbert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=40,
    )

tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier

In [18]:
model.eval()
# model()
input = tokenizer(["this is a test"], return_tensors='pt')
model(**input)


SequenceClassifierOutput(loss=None, logits=tensor([[ 0.0319, -0.1928, -0.1442, -0.0052, -0.0317,  0.0454,  0.0905,  0.0166,
          0.0020, -0.0565,  0.0201,  0.1488,  0.1418,  0.1372, -0.0034,  0.2713,
         -0.1397, -0.1612,  0.1082, -0.0883, -0.0179, -0.0782,  0.0019, -0.0383,
          0.0440, -0.0030,  0.0074, -0.1221, -0.1516, -0.1948,  0.0848,  0.0175,
          0.0493, -0.1257,  0.0237, -0.1884,  0.1570,  0.0039, -0.0843, -0.0045]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [79]:
session = Session(boto3.session.Session(region_name="eu-west-3"))


with load_run(
    sagemaker_session=session,
    # run_name="pipelines-7tlg1lriewey-train-model-KoqVPNiyFi",
    # experiment_name="training-pipeline",
    ) as run:
    run.log_metric(name="training-loss", value=100.0, step=1)

RuntimeError: Failed to load a Run object. Please make sure a Run object has been initialized already.

In [84]:
session.describe_training_job("pipelines-shv3uauomvc3-train-model-IekbsOvyGC")

{'TrainingJobName': 'pipelines-shv3uauomvc3-train-model-IekbsOvyGC',
 'TrainingJobArn': 'arn:aws:sagemaker:eu-west-3:101436505502:training-job/pipelines-shv3uauomvc3-train-model-IekbsOvyGC',
 'ModelArtifacts': {'S3ModelArtifacts': 's3://sagemaker-eu-west-3-101436505502/model/pipelines-shv3uauomvc3-train-model-IekbsOvyGC/output/model.tar.gz'},
 'TrainingJobStatus': 'Failed',
 'SecondaryStatus': 'Failed',
 'FailureReason': 'AlgorithmError: ExecuteUserScriptError:\nExitCode 1\nErrorMessage "raise RuntimeError(\n RuntimeError: Not able to fetch RunName in ExperimentConfig of the sagemaker job. Please make sure the ExperimentConfig is correctly set."\nCommand "/opt/conda/bin/python3.8 train.py --batch_size 10 --epoch_count 1", exit code: 1',
 'HyperParameters': {'batch_size': '10',
  'epoch_count': '1',
  'sagemaker_container_log_level': '20',
  'sagemaker_program': '"train.py"',
  'sagemaker_region': '"eu-west-3"',
  'sagemaker_submit_directory': '"s3://sagemaker-eu-west-3-101436505502/tra

In [2]:
class dummy:
    
    def __init__(self) -> None:
        self.account="101436505502"
        self.region="eu-west-3"
        self.pipeline_name="training-pipeline"

In [16]:
args = dummy()

region = boto3.Session(region_name=args.region).region_name

sagemaker_session = sagemaker.session.Session()

iam = boto3.client("iam")
role = iam.get_role(RoleName=f"{args.account}-sagemaker-exec")['Role']['Arn']

print(role)
default_bucket = sagemaker_session.default_bucket()
image_uri = f"{args.account}.dkr.ecr.{args.region}.amazonaws.com/{args.pipeline_name}:latest"
training_image_uri = f"{args.account}.dkr.ecr.{args.region}.amazonaws.com/training-image:latest"

model_path = f"s3://{default_bucket}/model"
data_path = f"s3://{default_bucket}/data"
model_package_group_name = f"{args.pipeline_name}ModelGroup"
pipeline_name = args.pipeline_name

huggingface_instance_type = "ml.m5.large"#"ml.g4dn.xlarge"
pytorch_version = "1.6.0"#"1.6"
transformers_version = "4.6.1"#"4.4"


arn:aws:iam::101436505502:role/101436505502-sagemaker-exec


In [17]:
estimator = Estimator(
    instance_type=huggingface_instance_type,
    instance_count=1,
    entry_point="steps/train.py",
    # sagemaker_session=sagemaker_session,
    role=role,
    output_path=model_path,
    base_job_name="train-estimator",
    image_uri=training_image_uri,
    py_version=None,
)

estimator.fit()

INFO:sagemaker:Creating training-job with name: train-estimator-2023-03-20-14-25-22-860


2023-03-20 14:25:23 Starting - Starting the training job..

KeyboardInterrupt: 

In [ ]:
estimator.hyperparameters()

{'sagemaker_submit_directory': '"s3://sagemaker-eu-west-3-101436505502/train-estimator-2023-03-20-10-16-00-001/source/sourcedir.tar.gz"',
 'sagemaker_program': '"train.py"',
 'sagemaker_container_log_level': '20',
 'sagemaker_job_name': '"train-estimator-2023-03-20-10-16-00-001"',
 'sagemaker_region': '"eu-west-3"'}

In [ ]:
estimator.CONTAINER_CODE_CHANNEL_SOURCEDIR_PATH

'/opt/ml/input/data/code/sourcedir.tar.gz'

In [ ]:
estimator.environment